## Align and stack previously reduced science data

### Written and revised (2016-2019) by J. Lowenthal, M. Petersen, K. Ward-Duong 

#### Please use your own reduced B, V, and R band images for this lab; if needed, backups are otherwise located at the following link:
https://drive.google.com/drive/u/1/folders/1GxsCM49WmZqgCrJ65KtAY965mrxe_zrB

____
### Lab Group Names:
___

In [ ]:
# Define the list of target(s) and the filter(s) you would like to align.
targs =['NGC6853']

filters=['Blue','Visual','Red']  # In order of increasing wavelength: BGR, not RGB

# Provide the path to your data directory, the folder with your individual reduced science images:
# The path format should be the absolute path, e.g., "/Users/StudentName/AST337/MyData" (note there is no '/' at the end!)
datadir =

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import astropy
from astropy.io import fits
import matplotlib.cm as cm
import scipy.signal
import glob
import os
from astropy.stats import sigma_clip
import time

### 1) Function definition to calculate cross-correlation between two images and measure x,y offsets.

In [ ]:
def cross_image(im1, im2, **kwargs):
    """
    Replace this with your own docstring that describes the inputs and methods used in the cross_image function. 
    Also complete any unfinished code and unfinished comments below.
    """
    
    # The type cast into 'float' is to avoid overflows:
    im1_gray = im1.astype('float')
    im2_gray = im2.astype('float')

    # Enable a trimming capability using keyword argument option.
    if 'boxsize' in kwargs:
        im1_gray = im1_gray[0:kwargs['boxsize'],0:kwargs['boxsize']]
        im2_gray = im2_gray[0:kwargs['boxsize'],0:kwargs['boxsize']]

    # Subtract the averages of im1_gray and im2_gray from their respective arrays -- cross-correlation
    # works better that way.
    # Complete the following two lines:
    im1_gray -= 
    im2_gray -= 

    # Calculate the correlation image using fast Fourier transform (FFT)
    # Note the flipping of one of the images (the [::-1]) - this is how the convolution is done.
    corr_image = scipy.signal.fftconvolve(im1_gray, im2_gray[::-1,::-1], mode='same')
    
    # To determine the location of the peak value in the cross-correlated image, complete the line below,
    # using np.argmax on the correlation image:
    peak_corr_index = 

    # Find the peak signal position in the cross-correlation -- this gives the shift between the images.
    corr_tuple = np.unravel_index(peak_corr_index, corr_image.shape)
    
    # Calculate shifts (not cast to integer, but could be).
    xshift = corr_tuple[0] - corr_image.shape[0]/2.
    yshift = corr_tuple[1] - corr_image.shape[1]/2.

    return xshift,yshift

In [ ]:
# Use this cell to look up the doc strings for any unfamiliar functions, e.g.,
scipy.signal.fftconvolve?

# You can also read more and see examples of Fourier transforms applied to images here:
# https://www.cs.unm.edu/~brayer/vision/fourier.html

### Questions:

What does np.unravel_index do?  
**Answer**: 

What kinds of python objects are corr_tuple and corr_image? That is, what kinds of objects are the outputs of scipy.signal.fftconvolve and np.unravel_index?  
**Answer**: 

In [ ]:
# Extra cell for testing code, looking up docstrings, etc:


### 2) Definition to shift ("roll" pixels in) image.

In [ ]:
def shift_image(image,xshift,yshift):
    '''
    Replace this with your own docstring that describes the inputs and operations applied in the shift_image function. 
    '''
    # Note that this will not do any trimming, 
    # so we'll want to  trim later the edges of the image using the maximum shift.
    return np.roll(np.roll(image,int(yshift),axis=1), int(xshift), axis=0)

### Questions:
How does np.roll shift the image?  
**Answer**: 

Why might we want the ability to trim a shifted image?  
**Answer**: 

In [ ]:
# Cycle through list of targets:

for targname in targs:
    print(' ')
    print('-----------------------------')      
    print('target: ', targname)
    print('-----------------------------')      

    # Using glob, make list of all reduced images of current target in all filters.
    # Complete the following line to create a list of the correct images to be shifted (use wildcards!):
    imlist = 
    
    # Check to make sure that your new list has the right files:
    print("All files to be aligned: \n", imlist)
    print('\n') # adding some space to the print statements, '/n' means new line
    
    # Open first image = master image; all other images of same target will be aligned to this one.
    im1,hdr1 = fits.getdata(imlist[0],header=True)
    print("Aligning all images to:", imlist[0])
    
    print('\n') # adding some space to the print statements

    # What is the following for loop doing?
    # Your answer:
    
    xshifts = {}
    yshifts = {}
    for index,filename in enumerate(imlist):
        im,hdr = fits.getdata(filename,header=True)
        xshifts[index], yshifts[index] = cross_image(im1, im, boxsize=1000)
        print("Shift for image", index, "is", xshifts[index], yshifts[index])

    # Calculate trim edges of new median stacked images so all stacked images of each target have same size 
    max_x_shift = int(np.max([xshifts[x] for x in xshifts.keys()]))
    max_y_shift = int(np.max([yshifts[x] for x in yshifts.keys()]))

    print('   Max x-shift={0}, max y-shift={1} (pixels)'.format(max_x_shift,max_y_shift))


    # Cycle through list of filters
    for filtername in filters:
        # Write a for-loop + if-statement to create a list of FITS files matching *only* the selected filter:
        scilist = []
        for fitsfile in imlist:
 
        
        if len(scilist) < 1:
            print("Warning! No files in scilist. Your path is likely incorrect.")
            break
        
        # Complete the print statement below including the filename & ensuring each scilist entry has the right filter:
        for fitsfile in scilist:
            print(
        
        nfiles = len(scilist)
        print('Stacking ', nfiles, filtername, ' science frames')

        # Define new array with same size as master image
        image_stack = np.zeros([im1.shape[0],im1.shape[1],len(scilist)])

        # Now that we have created an "empty" array, what is the following for loop doing?
        # Your answer: 
        
        xshifts_filt = {}
        yshifts_filt = {}
        for index,filename in enumerate(scilist):
            im,hdr = fits.getdata(filename,header=True)
            xshifts_filt[index], yshifts_filt[index] = cross_image(im1, im, boxsize=1000)
            image_stack[:,:,index] = shift_image(im,xshifts_filt[index], yshifts_filt[index])

        # Complete the line below to take the median of the image stack (median combine the stacked images);
        # Be careful to use the correct 'axis' keyword in the np.median function!
        median_image = np.median(

        # Sets the new image boundaries
        if (max_x_shift > 0) & (max_y_shift > 0): # don't apply cut if no shift!
            median_image = median_image[max_x_shift:-max_x_shift,max_y_shift:-max_y_shift]

        # Make a new directory in your datadir for the new stacked fits files
        if os.path.isdir(datadir + '/Stacked') == False:
            os.mkdir(datadir + '/Stacked')
            print('\n Making new subdirectory for stacked images:', datadir + '/Stacked \n')
            
        
        # Save the final stacked images into your new folder:
        fits.writeto(datadir + '/Stacked/' + targname + '_' + filtername + 'stack.fits', median_image, overwrite=True)
        print('   Wrote FITS file ',targname+'_'+filtername+'stack.fits', 'in ',datadir + '/Stacked/','\n')
        
print('\n Done stacking!')
